In [40]:
import os, logging, sys, pickle
import networkx as nx
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)
import pandas as pd

In [45]:
df = pd.read_csv(all_data_dirs['pipelines-N'])

In [2]:
        all_data_dirs = {
            'cities-N':              os.path.join(os.getcwd(),'..','results_backup','output','cities_nodes_dataframe.csv'),
            'pipelines-cities':      os.path.join(os.getcwd(),'..','results_backup','output','cities_pipelines_edge_dataframe.csv'),
            'ports-cities':          os.path.join(os.getcwd(),'..','results_backup','output','cities_ports_edge_dataframe.csv'),
            'railways-cities':       os.path.join(os.getcwd(),'..','results_backup','output','cities_railways_edge_dataframe.csv'),
            'coalmines-railways':    os.path.join(os.getcwd(),'..','results_backup','output','coal_mine_railway_edge_dataframe.csv'),
            'coalmines-N':           os.path.join(os.getcwd(),'..','results_backup','output','coal_mines_nodes_dataframe.csv'),
            'lng-N':                 os.path.join(os.getcwd(),'..','results_backup','output','lng_nodes_dataframe.csv',  ),
            'lng-pipelines':         os.path.join(os.getcwd(),'..','results_backup','output','lng_pipeline_edge_dataframe.csv'),
            'lng-shipping':          os.path.join(os.getcwd(),'..','results_backup','output','lng_shipping_route_edge_dataframe.csv'),
            'oilfields-pipelines':   os.path.join(os.getcwd(),'..','results_backup','output','oil_field_edge_dataframe.csv'),
            'oilfields-N':           os.path.join(os.getcwd(),'..','results_backup','output','oil_field_nodes_dataframe.csv'),
            'pipelines-pipelines':   os.path.join(os.getcwd(),'..','results_backup','simplify','pipeline_edge_dataframe.csv'),
            'pipelines-N':           os.path.join(os.getcwd(),'..','results_backup','output','pipeline_node_dataframe.csv'),
            'ports-N':               os.path.join(os.getcwd(),'..','results_backup','output','port_node_dataframe.csv',  ),
            'ports-pipelines':       os.path.join(os.getcwd(),'..','results_backup','output','port_pipeline_edge_dataframe.csv'),
            'ports-shipping':        os.path.join(os.getcwd(),'..','results_backup','output','port_ship_edge_dataframe.csv'),
            'powerstn-N':            os.path.join(os.getcwd(),'..','results_backup','output','power_station_nodes_dataframe.csv'),
            'powerstn-pipelines':    os.path.join(os.getcwd(),'..','results_backup','output','power_station_pipeline_edge_dataframe.csv'),
            'powerstn-railways':     os.path.join(os.getcwd(),'..','results_backup','output','power_station_railway_edge_dataframe.csv'),
            'procplant-N':           os.path.join(os.getcwd(),'..','results_backup','output','processing_plant_nodes_dataframe.csv'),
            'procplant-pipelines':   os.path.join(os.getcwd(),'..','results_backup','output','processing_plant_pipeline_edge_dataframe.csv'),
            'railways-railways':     os.path.join(os.getcwd(),'..','results_backup','simplify','railway_edge_dataframe.csv'),
            'railways-N':            os.path.join(os.getcwd(),'..','results_backup','output','railway_nodes_dataframe.csv'),
            'refineries-N':          os.path.join(os.getcwd(),'..','results_backup','output','refinery_nodes_dataframe.csv'),
            'refineries-pipelines':  os.path.join(os.getcwd(),'..','results_backup','output','refinery_pipeline_edge_dataframe.csv'),
            'shipping-shipping':     os.path.join(os.getcwd(),'..','results_backup','output','shipping_edge_dataframe.csv'),
            'shipping-N':            os.path.join(os.getcwd(),'..','results_backup','output','shipping_node_dataframe.csv'),
            'wellpads-N':            os.path.join(os.getcwd(),'..','results_backup','output','well_pad_nodes_dataframe.csv'),
            'wellpads-pipelines':    os.path.join(os.getcwd(),'..','results_backup','output','well_pad_pipeline_edge_dataframe.csv'),
        }

In [21]:
recipe = [
        {'name':'wellpads-pipelines',       'reverse':False,    'dup_1':False, 'dup_2':False,   'desc':'add wellpads -> pipelines'},
        {'name':'oilfields-pipelines',      'reverse':False,    'dup_1':False, 'dup_2':False,   'desc':'add fields -> pipelines'},
        {'name':'pipelines-pipelines',      'reverse':False,    'dup_1':False, 'dup_2':False,   'desc':'add pipelines -> pipelines'},
        {'name':'pipelines-pipelines',      'reverse':True,     'dup_1':False, 'dup_2':False,   'desc':'add pipelines <- pipelines'},
        {'name':'ports-pipelines',          'reverse':False,    'dup_1':False, 'dup_2':False,   'desc':'add pipelines -> ports'},
        {'name':'ports-pipelines',          'reverse':True,     'dup_1':False, 'dup_2':False,   'desc':'add pipelines <- ports'},
        {'name':'ports-shipping',           'reverse':False,    'dup_1':False, 'dup_2':False,   'desc':'add ports -> shipping_lanes'},
        {'name':'ports-shipping',           'reverse':True,     'dup_1':False, 'dup_2':False,   'desc':'add ports <- shipping_lanes'},
        {'name':'shipping-shipping',        'reverse':False,    'dup_1':False, 'dup_2':False,   'desc':'add shipping_lanes -> shipping_lanes'},
        {'name':'shipping-shipping',        'reverse':True,     'dup_1':False, 'dup_2':False,   'desc':'add shipping_lanes <- shipping_lanes'},
        {'name':'refineries-pipelines',     'reverse':True,     'dup_1':False, 'dup_2':False,   'desc':'add pipelines -> refineries'},
        {'name':'refineries-pipelines',     'reverse':False,    'dup_1':False, 'dup_2':True,    'desc':'add refineries -> pipelines_2'},
        {'name':'ports-pipelines',          'reverse':True,     'dup_1':True,  'dup_2':True,    'desc':'add pipelines_2 -> ports_2 '},
        {'name':'ports-pipelines',          'reverse':False,    'dup_1':True,  'dup_2':True,    'desc':'add pipelines_2 <- ports_2 '},
        {'name':'ports-shipping',           'reverse':False,    'dup_1':True,  'dup_2':True,    'desc':'add ports_2 -> shipping_lanes_2'},
        {'name':'ports-shipping',           'reverse':True,     'dup_1':True,  'dup_2':True,    'desc':'add ports_2 <- shipping_lanes_2'},
        {'name':'shipping-shipping',        'reverse':False,    'dup_1':True,  'dup_2':True,    'desc':'add shipping_lanes_2 -> shipping_lanes_2'},
        {'name':'shipping-shipping',        'reverse':True,     'dup_1':True,  'dup_2':True,    'desc':'add shipping_lanes_2 <- shipping_lanes_2'},
        {'name':'pipelines-cities',         'reverse':False,    'dup_1':True, 'dup_2':False,    'desc':'add pipelines_2 -> cities'},
        {'name':'powerstn-pipelines',       'reverse':True,     'dup_1':False, 'dup_2':True,    'desc':'add pipelines_2 -> power_stn'},
        ]

In [22]:
dfs = {}
datasets = list(set([step['name'] for step in recipe]))
for ds in datasets:
    print (ds)
    dfs[ds] = pd.read_csv(all_data_dirs[ds])

pipelines-cities
ports-shipping
pipelines-pipelines
ports-pipelines
shipping-shipping
wellpads-pipelines
refineries-pipelines
powerstn-pipelines
oilfields-pipelines


In [32]:
dfs['ports-pipelines']['impedance'] = 1

In [33]:
G = nx.DiGraph()

In [35]:
add_steps = [    
    {'name':'pipelines-pipelines',      'reverse':False,    'dup_1':True,  'dup_2':True,    'desc':'add pipelines_2 -> pipelines_2'},
    {'name':'pipelines-pipelines',      'reverse':True,     'dup_1':True,  'dup_2':True,    'desc':'add pipelines_2 <- pipelines_2'}]

In [36]:
for step in add_steps:
            logger.info(f'doing step {step["desc"]}...')
            dup_strs = ['','']
            start_col = [cc for cc in dfs[step['name']].columns if 'START_ID' in cc][0]
            end_col = [cc for cc in dfs[step['name']].columns if 'END_ID' in cc][0]
            order = [start_col,end_col]
            if step['dup_1']:
                dup_strs[0]='_B'
            if step['dup_2']:
                dup_strs[1]='_B'
            if step['reverse']:
                order.reverse()
                dup_strs.reverse()
            print (step['name'],order)

            G.add_edges_from([(r[0]+dup_strs[0],r[1]+dup_strs[1],{'z':r[2]}) for r in dfs[step['name']].loc[:,order+['impedance']].values.tolist()])

INFO:__main__:doing step add pipelines_2 -> pipelines_2...
pipelines-pipelines ['StartNodeId:START_ID(PipelineNode)', 'EndNodeId:END_ID(PipelineNode)']
INFO:__main__:doing step add pipelines_2 <- pipelines_2...
pipelines-pipelines ['EndNodeId:END_ID(PipelineNode)', 'StartNodeId:START_ID(PipelineNode)']


In [37]:
len(G.nodes)

8996809

In [38]:
len(G.edges)

48427013

In [39]:
print (nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 8996809
Number of edges: 48427013
Average in degree:   5.3827
Average out degree:   5.3827
